In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

model=model=ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest', temperature=0)



                   TypedDict


In [3]:
from typing import TypedDict, Annotated

class Review(TypedDict):
    summary: Annotated[str, "A brief summary of the review"]
    sentiment: Annotated[str, "Return Sentiment of the review either negative, postive or neutral"]

structured_model= model.with_structured_output(Review)


result = structured_model.invoke("""The hardware is greate, but the software feels bloated. There are too many pre-installed apps that I don't use. Also, the UI looks outdated compared to other brands""")
print(result)

print(type(result))


{'sentiment': 'negative', 'summary': "The hardware is great, but the software feels bloated. There are too many pre-installed apps that I don't use. Also, the UI looks outdated compared to other brands"}
<class 'dict'>


using of optionl and Annotated

In [4]:
from typing import TypedDict, Annotated, Optional

class Review(TypedDict):
    key_theme: Annotated[list[str], "Write down all the key theme discussed in the review in a list"]
    
    summary: Annotated[str, "A brief summary of the review"]

    sentiment: Annotated[str, "Return Sentiment of the review either negative, postive or neutral"]

    pros: Annotated[Optional[list[str]], "Write down all the pros inside a list"]

    cons: Annotated[Optional[list[str]],"Write down all the cons inside a list"]

structured_model= model.with_structured_output(Review)

text= """ 
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Review by Nitish Singh
"""

result = structured_model.invoke(text)

print(result)


None


                            Pydantic

how pydantic works


In [5]:
from pydantic import BaseModel, Field
from typing import Optional

class Student(BaseModel):
    name: str= "Aman"
    age: Optional[int] = None
    cgpa : float = Field(gt=0, lt=10, default=5, description="A decimal value representing the cgpa of the student")

new_student={'name':'Aman', 'cgpa':8}
# new_student={'name':32}
student=Student(**new_student)

student_dict= dict(student)
student_json= student.model_dump_json()
print(student_json)

print(student_dict)


{"name":"Aman","age":null,"cgpa":8.0}
{'name': 'Aman', 'age': None, 'cgpa': 8.0}


Using with Structure Output

In [8]:
from pydantic import BaseModel, Field

class Review(BaseModel):
    key_themes: list[str] = Field(description="Write down all the themes discussed in the review")

    summary:  str = Field(description="Write down the output summary of the review")

    sentiment: str = Field(description="return Sentiment of the review either negative, postive or neutral")

    pros: Optional[list[str]]= Field(default=None, description="Write down all the pros in the review")

    cons: Optional[list[str]]= Field(default=None, description="Write down all the cons in the review")

structured_model= model.with_structured_output(Review)


result = structured_model.invoke("""
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Review by Nitish Singh
""")
result_dict=dict(result)
print(result_dict)


{'key_themes': ['Processor', 'Battery', 'Camera', 'S-Pen', 'Size and Weight', 'Bloatware', 'Price'], 'summary': 'The Samsung Galaxy S24 Ultra is a powerful phone with a great camera and long battery life, but it is expensive and heavy.', 'sentiment': 'positive', 'pros': ['Fast processor', 'Long battery life', 'Excellent camera', 'Good S-Pen integration'], 'cons': ['Heavy and large', 'Bloatware', 'Expensive']}


                           JSON 

In [9]:
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}
structured_model= model.with_structured_output(json_schema)


result = structured_model.invoke("""
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Review by Nitish Singh
""")

print(result)


Key 'parameters' is not supported in schema, ignoring


{'name': 'Nitish Singh', 'sentiment': 'pos', 'pros': 'The Snapdragon 8 Gen 3 processor is very fast. The 5000mAh battery lasts a full day even with heavy use. The 200MP camera is excellent, especially in night mode. The S-Pen integration is a nice touch.', 'cons': 'The phone is heavy and bulky, making it uncomfortable for one-handed use. It also comes with bloatware and has a high price tag.', 'summary': 'A powerful phone with great camera and battery life, but it is expensive and a bit bulky.', 'key_themes': ['Processor', 'Battery', 'Camera', 'S-Pen', 'Size and Weight', 'Bloatware', 'Price']}
